# Simple neural network with PyTorch
## Digit recognizer challenge

In [1]:
import pandas as pd

In [2]:
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [16]:
X = torch.tensor(train.iloc[:, 1:].values, dtype=torch.float32) / 255.0  # Normalize pixel values to 0-1
X_test = torch.tensor(test.values, dtype=torch.float32) / 255.0
y = torch.tensor(train.iloc[:, 0].values, dtype=torch.long)  # Labels

In [10]:
# Create DataLoader
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

In [11]:
# Define the neural network
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden1_size, hidden2_size, num_classes):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden2_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

In [12]:
# Hyperparameters
input_size = 784  # 28x28 pixels
hidden1_size = 128
hidden2_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 10

# Initialize the network
model = SimpleNN(input_size, hidden1_size, hidden2_size, num_classes)

In [13]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
# Training loop
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(dataloader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Training finished!")

Epoch [1/10], Loss: 0.4598
Epoch [2/10], Loss: 0.1119
Epoch [3/10], Loss: 0.0581
Epoch [4/10], Loss: 0.0716
Epoch [5/10], Loss: 0.4523
Epoch [6/10], Loss: 0.0062
Epoch [7/10], Loss: 0.0089
Epoch [8/10], Loss: 0.1129
Epoch [9/10], Loss: 0.0136
Epoch [10/10], Loss: 0.0136
Training finished!


In [15]:
# Evaluate the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in dataloader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of the model on the training images: {100 * correct / total}%')

Accuracy of the model on the training images: 99.52380952380952%


In [17]:
# Make predictions
model.eval()
with torch.no_grad():
    predictions = model(X_test)
class_predictions = predictions.argmax(dim=1)

In [19]:
submission = pd.DataFrame({
    "ImageId": range(1, 28001),
    "Label": class_predictions
})
submission.to_csv('submission.csv', index=False)